# 多任务编程——多进程
## 理解多任务——理解操作系统的CPU上下文
- 多任务即是同一时间让系统执行多个任务，其中包括并发和并行两种方式。
### 并发与并行
- 并发：又称伪并行，是指一个时间段中有几个程序都处于已启动运行到运行完毕之间，且这几个程序都是在同一个处理机上运行，但任意时刻真正在工作的只有一个程序。比如说，一秒内切换了100个线程，就可以认为CPU的并发是100.单个CPU+多道技术即可实现
- 并行：指在任意时刻点上，在同一处理机上有多个程序在CPU上同时工作，每个程序独立运行，互不干扰，最大并行数量和CPU数量是一致的。可以理解为多CPU即多核运行
- 平常所说的是高并发而不是高并行，因为每台电脑的CPU数量有限，不可以增加
#### 并发的工作原理——CPU上下文
        CPU一个时间段只能运行单个任务，只不过在很短的时间内，CPU快速切换到不同的任务执行，造成一种多任务同时执行的错觉
        而CPU在切换到其他任务执行之前，为了确保切换任务之后还能够继续切换到原来的任务执行，并且看起来是一种连续的状态，就必须将任务的状态保持起来，以便恢复上个任务的执行状态，而该状态保存在CPU的寄存器和程序计数器中。寄存器：CPU内置的容量小，但速度极快的内存，用来保存程序的堆栈信息即数据段信息。程序计数器：保存程序的下一条指令的位置即代码段信息。
        所以，CPU上下文就是指CPU寄存器和程序计数器中保存的任务状态信息，CPU上下文切换就是把前一个任务的CPU上下文保存起来，然后加载下一个任务的上下文到这些寄存器和程序计数器，再跳转到程序计数器所指定的位置运行程序。
#### python中多任务的实现方法
- Python程序默认都是执行单任务的程序，也就是只有一个线程，若要执行多任务，办法有以下两种：
    - 第一种：启动多个进程，每个进程只有一个线程，但多个进程可以一块执行多个任务
    - 第二种：启动一个进程，在这个进程内启动多个线程
    - 第三种：启动多个进程，每个进程再启动多个线程。此方法目前不常用


### 同步与异步
- 同步：(注意同步和异步只是针对于I/O操作来讲的）值调用IO操作时，必须等待IO操作完成后才开始新的的调用方式。
- 异步：指调用IO操作时，不必等待IO操作完成就开始新的的调用方式。不过等到IO操作完成时，需要告知调用者自己已经完成，告知方式有：
    - 1、状态：调用者监听被调用者的状态，即时不时的检查被调用者的状态。效率低
    - 2、通知：当被调用者执行完毕后，被调用者通知调用者自己执行完毕
    - 3、回调：当被调用者执行完毕，它调用由调用者提供给他的回调函数

### 阻塞与非阻塞
- 阻塞：阻塞调用是指调用结果返回之前，当前线程会被挂起。调用线程只有在得到结果之后才会返回。
- 非阻塞：非阻塞调用指在不能立刻得到结果之前，该调用不会阻塞当前线程。

- 注意：同步执行一般都会有阻塞，但也有可能没阻塞；异步执行也有可能有阻塞，也可能没有阻塞。

# 全局解释器锁（GIL）
- 上面说到，Python只能默认执行单任务，究其原因就是全局解释器的存在
- 其全称为Global Interpreter Lock(全局解释器锁）
            设计之初是为了数据安全所做的决定：某个线程想要执行，必须拿到GIL,否则就不允许进入CPU执行。所以可以把它看成是通行证，在一个进程中，GIL只有一个、只有cpython中有GIL，pypy和jpython中没有它。
- python代码的执行时由Python虚拟机进行控制
- 在主循环中只能够有一个控制线程在运行

# 多进程
- 进程是操作系统中进行资源分配和调度的最小单位，也就是基本单位。一个正在运行的程序就是一个进程，一个程序运行至少需要一个进程来进行资源分配和调度。
- 值得注意的是：如果在window操作系统下，所有和进程相关的代码都必须放在__name__ == "__main__"下面，否则会报错

## 多进程的创建方法
### 方法一：fork()函数
- 使用fork()函数创建（仅在Linux或Unix系统生效）
- 在Unix/Linux系统中，提供了一个fork()函数调用，相较于普通函数调用一次，返回一次的机制，fork()调用一次，返回两次，具体表现为操作系统自动把当前进程(称为父进程)复制了一份(称为子进程)，然后分别在父进程和子进程内返回。
- 子进程永远返回0，而父进程返回子进程的ID，这样一个父进程可以轻松fork出很多子进程。且父进程会记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。
- 通过fork调用这种方法，一个进程在接到新任务时就可以复制出一个子进程来处理新任务，例如nginx就是由父进程(master process)监听端口，再fork出子进程(work process)来处理新的http请求。
- Windows没有fork调用，所以在window pycharm上运行以上代码无法实现以上效果。

### 方法二：multiprocessing.Process
- 使用python自带的库模块：multiprocessing模块，实例化一个multiprocessing.Process的对象，并传入一个初始化函数对象（initial function )作为新建进程执行入口；
- 本质来说，multiprocessing不是一个模块，而是一个包
- 其包含内容大致可以分为四个部分：
    - 创建多进程部分：multiprocessing.Process,subprocess模块（不是multiprocessing中的模块）
    - 多进程同步部分：进程锁multiprocessing.Lock
    - 进程池部分：multiprocessing.Pool
    - 多进程数据共享部分:multiprocessing.Queue, multiprocessing.Pipe
    
### 方法三：继承multiprocessing.Process()
- 要构造init函数（非必须）
- 重写run函数（必须）

### 方法四：subprocess模块
- 目的是允许产生一个新的进程，连接输入(input)/输出(output)/错误(error)的管道，返回子进程的返回值，替代老模块如os.system, os.spawn
- Sunprocess模块可以在程序执行过程中，调用外部的程序。
- 比如我们可以在python程序中打开记事本，打开cmd，或者在某个时间点关机

### 方法五：进程池方法


### 获取进程编号
- 1.获取当前进程编号：os.getpid()
- 2.获取当前父进程编号：os.getppid()

### 设置守护主进程：
- 1.主进程默认情况下会等到所有子进程结束才结束，即父进程的代码执行完毕但子进程的代码未执行完毕，所以程序会一直等待子进程执行完毕。
- 2.设置 work_process.daemon = True,则主进程结束，子进程都会结束
- 3.守护进程将无法再创建子进程

### Process对象的join方法
- 使用Process创建了子进程，调用start方法后，父子进程会砸死各自的进程中不断的执行代码，有时候如果想等待子进程执行完毕后再执行下面的代码，此时可调用join方法
- join方法可传入timeout参数

# process类的其他方法
- 构造方法：
        Process([group [, target [, name [, args [, kwargs]]]]])
          group: 线程组 
          target: 要执行的方法
          name: 进程名
          args/kwargs: 要传入方法的参数

- 实例方法：
        is_alive()：返回进程是否在运行,bool类型。
        join([timeout])：阻塞当前上下文环境的进程程，直到调用此方法的进程终止或到达指定的timeout（可选参数）。
        start()：进程准备就绪，等待CPU调度
        run()：strat()调用run方法，如果实例进程时未制定传入target，这star执行t默认run()方法。
        terminate()：不管任务是否完成，立即停止工作进程

- 属性：
        daemon：和线程的setDeamon功能一样
        name：进程名字
        pid：进程号

In [1]:
import threading, multiprocessing
print(multiprocessing.__all__)
print("------------------")
print(threading.__all__)


['Array', 'AuthenticationError', 'Barrier', 'BoundedSemaphore', 'BufferTooShort', 'Condition', 'Event', 'JoinableQueue', 'Lock', 'Manager', 'Pipe', 'Pool', 'Process', 'ProcessError', 'Queue', 'RLock', 'RawArray', 'RawValue', 'Semaphore', 'SimpleQueue', 'TimeoutError', 'Value', 'active_children', 'allow_connection_pickling', 'cpu_count', 'current_process', 'freeze_support', 'get_all_start_methods', 'get_context', 'get_logger', 'get_start_method', 'log_to_stderr', 'reducer', 'set_executable', 'set_forkserver_preload', 'set_start_method']
------------------
['get_ident', 'active_count', 'Condition', 'current_thread', 'enumerate', 'main_thread', 'TIMEOUT_MAX', 'Event', 'Lock', 'RLock', 'Semaphore', 'BoundedSemaphore', 'Thread', 'Barrier', 'BrokenBarrierError', 'Timer', 'ThreadError', 'setprofile', 'settrace', 'local', 'stack_size']


In [2]:
import os
from random import randint
import time

def download(filename):
    print("进程号是%s"% os.getpid())
    print("现在开始下载{}".format(filename))
    time.sleep(1)
    
def runtask():
    start_time = time.time()
    download("新白娘子传奇")
    download("哪吒")
    stop_time = time.time()
    print("下载耗时{}".format(stop_time-start_time))
    
if __name__ == "__main__":
    runtask()

进程号是16336
现在开始下载新白娘子传奇
进程号是16336
现在开始下载哪吒
下载耗时2.0022084712982178


In [3]:
# 不设置join，则会优先执行完主进程中的代码
import time
import os
import multiprocessing


def download(filename):
    print("进程号是%s" % os.getpid())
    print("父进程是{}".format(os.getppid()))
    print("现在开始下载{}".format(filename))
    time.sleep(1)


def main():
    start_time = time.time()
    p1 = multiprocessing.Process(target=download, args=("新白娘子传奇",))
    p2 = multiprocessing.Process(target=download, args=("西游记",))
    p1.start()
    p2.start()
    end_time = time.time()
    print("总共耗时：{}".format(end_time - start_time))


if __name__ == "__main__":
    main()
# 运行有bug，不知道为啥，可以使用pycharm

总共耗时：0.1216726303100586


In [4]:
# 调用Process模拟开启两个进程
import time
import os
import multiprocessing

def download(filename):
    print("进程号是%s"% os.getpid())
    print("父进程是{}".format(os.getppid()))
    print("现在开始下载{}".format(filename))
    time.sleep(1)
    
def main():
    start_time = time.time()
    p1 = multiprocessing.Process(target=download, args=("新白娘子传奇", ))
    p2 = multiprocessing.Process(target=download, args=("西游记", ))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end_time = time.time()
    print("总共耗时：{}".format(end_time - start_time))

if __name__ == "__main__":
    main()
# 运行有bug，不知道为啥，可以使用pycharm

总共耗时：0.47228264808654785


In [5]:
# 继承multiprocessing.Process()
import multiprocessing
import os, time

class myProcess(multiprocessing.Process):
    def __init__(self):
        multiprocessing.Process.__init__(self)
        
    def run(self):
        print("子进程开始>>> pid={0}, ppid={1}".format(os.getpid(), os.getppid()))
        time.sleep(1)
        print("子进程终止>>> pid={}".format(os.getpid()))
        
def main():
    print("主进程开始{}".format(__name__))
    p1 = myProcess()
    p2 = myProcess()
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print("程序运行完毕")
    
if __name__ == "__main__":
    main()

主进程开始__main__
程序运行完毕


In [6]:
# subprocess模块
import subprocess
import os
a = os.system("df - Th")
print(a)
#subprocess.run(["df", "-h"])

1



# 进程同步：进程锁
- 加锁可以保证多个进程修改同一块数据时，同一时间只能有一个任务可以进行修改，即串行的修改，牺牲了速度却保证了数据安全。进程锁的问题：

        效率低（共享数据基于文件，而文件是硬盘上的数据）
        需要自己加锁处理

In [7]:
# 进程锁
from multiprocessing import Process, Lock
import os


def f(l, i):
    l.acquire()
    try:
        print(os.getpid())
        print('hello world', i)
    finally:
        l.release()


if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

# 进程池
- multiprocessing模块中有一个类Pool，这个类相当于一个池子，专门用来存储进程。Pool的__init__可以传递一个参数，这个参数指定这个进程池中同一时刻最多只能拥有多少个进程。并且，在使用进程池，父进程不会等待子进程池中的子进程执行完毕后退出，而是当父进程中的代码执行完毕后立即退出
- 当有新的请求提交到Pool中时，如果池还没满，就会创建一个新的进程执行该请求，但若进程中的进程数已达到最大值，那么直到池中某个进程结束才会用该进程执行该请求。

# 进程池Pool常用方法
- apply_async(func, args=(), kwds={}, callback=None, error_callback=None)
    - 异步非阻塞式的方法，子进程与父进程并行执行
    - callback:回调函数，若callback= func, 子进程执行完后，才会执行callback中的函数，否则callback不执行（而且callback中的函数是由父进程来执行了）
                进程池中任何一个任务一旦处理完了，就立即告知主进程：我好了额，你可以处理我的结果了。主进程则调用一个函数去处理该结果，该函数即回调函数
- apply(func, args, kwds)
    - 同步阻塞式方法，子进程与父进程同步执行
    - 同步与异步的的区分
        - 同步
            - 同步是指一个进程执行结束后才能执行另外一个进程，父进程是最后执行完毕的
        - 异步
            - 异步是指进程在执行某个请求时，不管其他的进程的状态，这个进程就执行后续操作；当有消息返回时系统会通知进程进行处理，这样可以提高执行的效率。所以异步执行父进程不会等待子进程，所以需要使用join()
- terminate():不管任务是否完成， ⽴即关闭进程池
- join():主进程等待所有子进程执行完毕，必须在close或terminete之后
- close():关闭Pool， 使其不再接受新的任务；等待所有进程结束才关闭线程池

In [ ]:
# 异步非阻塞执行apply_async()
# 同步阻塞速度很慢，建议弃用
from multiprocessing import Pool
from random import randint
import os,time

def download(taskname):
    print("进程号是{}".format(os.getpid()))
    downloadtime = randint(1,3)
    print("现在开始下载{}".format(taskname))
    time.sleep(downloadtime)

def runtask():
    start_time = time.time()
    pool = Pool(4)
    for task in range(8):
        pool.apply_async(download, args=(task, ))
    pool.close()
    pool.join()
    end_time = time.time()
    print("下载耗时：{}".format(end_time - start_time))
    
if __name__ == "__main__":
    runtask()

In [ ]:
# apply_async中的callback回调函数
from multiprocessing import Pool
import os, time, random

# 子进程任务函数
def download(f):
    print("进程中的进程：pid = {}, ppid = {}".format(os.getpid(), os.getppid()))
    for i in range(3):
        print(f, "--文件--%d"%i)
        time.sleep(1)
    return {"result":1, "info": "下载完成！"}

# 主进程调用回调函数
# 此处msg参数是从子进程任务函数return得到的字典
def alterUser(msg):
    print("callback function: pid = {}".format(os.getpid()))
    print("get result:", msg['info'])

if __name__ == "__main__":
    p = Pool(3)
    for arg in ['1111', '2222', '3333', '4444']:
        p.apply_async(download, args=(arg, ), callback=alterUser)
    p.close()
    p.join()
    print("程序结束")

# 进程间通信
- 用来解决进程锁存在的问题
- 在一个程序中，如果创建了一个子进程，那么这个子进程会拷贝一份当前进程所有的资源作为子进程的运行环境（因此子进程越多 ，拷贝复制越多，越消耗资源），也就是说，子进程中的变量可能和父进程一样，但其实 是另外一块内存区域了。它们之间的数据是不共享的
- 由于进程间数据是不共享的，所以不会出现多线程GIL带来的问题，多进程之间的通信通过队列Queue或管道Pipe来实现。

## 队列：multiprocessing.Queue()
- 使用方法和threading里面的Queue差不多
- 底层队列使用管道和锁实现
- Queue常用方法介绍
    - q = Queue([maxsize]) 
        - 创建共享的进程队列。maxsize是队列中允许的最大项数。如果省略此参数，则无大小限制。底层队列使用管道和锁实现。另外，还需要运行支持线程以便队列中的数据传输到底层管道中。 
    - Queue的实例方法：

    - q.get( [ block [ ,timeout ] ] ) 
        - 返回q中的一个项目。如果q为空，此方法将阻塞，直到队列中有项目可用为止。block用于控制阻塞行为，默认为True. 如果设置为False，将引发Queue.Empty异常（定义在Queue模块中）。timeout是可选超时时间，用在阻塞模式中。如果在制定的时间间隔内没有项目变为可用，将引发Queue.Empty异常。

    - q.get_nowait( ) 
        - 同q.get(block=False)方法。
    - q.put(item [, block [,timeout ] ] ) 
        - 将item放入队列。如果队列已满，此方法将阻塞至有空间可用为止。block控制阻塞行为，默认为True。如果设置为False，将引发Queue.Empty异常（定义在Queue库模块中）。timeout指定在阻塞模式中等待可用空间的时间长短。超时后将引发Queue.Full异常。
    - q.put_nowait():非阻塞的推入一条 消息，如果这个队列已经满了，那么会立马抛出异常，等价于q.put(block=False)
    - q.qsize() 
        - 返回队列中目前项目的正确数量。此函数的结果并不可靠，因为在返回结果和在稍后程序中使用结果之间，队列中可能添加或删除了项目。在某些系统上，此方法可能引发NotImplementedError异常。
    - q.empty() 
        - 如果调用此方法时 q为空，返回True。如果其他进程或线程正在往队列中添加项目，结果是不可靠的。也就是说，在返回和使用结果之间，队列中可能已经加入新的项目。
    - q.full() 
        - 如果q已满，返回为True. 由于线程的存在，结果也可能是不可靠的（参考q.empty（）方法）
    - q.close() 
        - 关闭队列，防止队列中加入更多数据。调用此方法时，后台线程将继续写入那些已入队列但尚未写入的数据，但将在此方法完成时马上关闭。如果q被垃圾收集，将自动调用此方法。关闭队列不会在队列使用者中生成任何类型的数据结束信号或异常。例如，如果某个使用者正被阻塞在get（）操作上，关闭生产者中的队列不会导致get（）方法返回错误。
    - q.cancel_join_thread() 
            - 不会在进程退出时自动连接后台线程。这可以防止join_thread()方法阻塞。
    - q.join_thread() 
        - 连接队列的后台线程。此方法用于在调用q.close()方法后，等待所有队列项被消耗。默认情况下，此方法由不是q的原始创建者的所有进程调用。调用q.cancel_join_thread()方法可以禁止这种行为。
        
### 解决发送信号结束问题：JoinableQueue
- 但队列允许项目的使用者通知生成者项目已经被成功处理。通知进程是使用共享的信号和条件变量来实现的。
- 除了与Queue对象相同的方法之外还具有：
    - q.task_done()：使用者使用此方法发出信号，表示q.get()的返回项目已经被处理。如果调用此方法的次数大于从队列中删除项目的数量，将引发ValueError异常
    - q.join():生产者调用此方法进行阻塞，直到队列中所有的项目均被处理。阻塞将持续到队列中的每个项目均调用q.task_done()方法为止，也就是队列中的数据全部被get拿走了。


## 管道：multiprocessing.Pipe（）
- 本质是数据传递而不是数据共享，管道有两道口，两个进程分别位于管道的两端，一端用来发送数据，一端用来接收数据。每端都有send()和recv()方法，如果两个进程试图在同一时间的同一端进行读取和写入，那么有可能会损坏管中的数据。
- 在进程间创建一条管道，并返回元组（conn1,conn2）,其中conn1和conn2是表示管道两端的Connection对象。默认控制下管道是双向的。如果将duplex设为False，conn1只能接受，conn2只能用于发送。(conn1, conn2) = multiprocessing.Pipe(duplex=True)
- 实例方法：
    - conn1.recv():接收conn2.send(obj)发送的对象。如果没有消息可接收，recv方法会一直阻塞。如果连接的另外一端已经关闭，那么recv方法会抛出EOFError。
    - conn1.send(obj):通过连接发送对象。obj是与序列化兼容的任意对象

    - conn1.close():关闭连接。如果conn1被垃圾回收，将自动调用此方法

    - conn1.fileno():返回连接使用的整数文件描述符

    - conn1.poll([timeout]):如果连接上的数据可用，返回True。timeout指定等待的最长时限。如果省略此参数，方法将立即返回结果。如果将timeout射成None，操作将无限期地等待数据到达。

    - conn1.recv_bytes([maxlength]):接收c.send_bytes()方法发送的一条完整的字节消息。maxlength指定要接收的最大字节数。如果进入的消息，超过了这个最大值，将引发IOError异常，并且在连接上无法进行进一步读取。如果连接的另外一端已经关闭，再也不存在任何数据，将引发EOFError异常。

    - conn.send_bytes(buffer [, offset [, size]])：通过连接发送字节数据缓冲区，buffer是支持缓冲区接口的任意对象，offset是缓冲区中的字节偏移量，而size是要发送字节数。结果数据以单条消息的形式发出，然后调用c.recv_bytes()函数进行接收    

    - conn1.recv_bytes_into(buffer [, offset]):接收一条完整的字节消息，并把它保存在buffer对象中，该对象支持可写入的缓冲区接口（即bytearray对象或类似的对象）。offset指定缓冲区中放置消息处的字节位移。返回值是收到的字节数。如果消息长度大于可用的缓冲区空间，将引发BufferTooShort异常。
- 如果是生产者或消费者中都没有使用管道的某个端点，就应将它关闭。这也说明了为何在生产者中关闭了管道的输出端，在消费者中关闭管道的输入端。
- 多消费者时调用需要加锁

## multiprocessing.Manager()
- 实现进程间数据共享。Manager()返回的manager对象会通过一个服务进程，来使其他进程通过代理的方式操作python对象。manager对象支持 list, dict, Namespace, Lock, RLock, Semaphore, BoundedSemaphore, Condition, Event, Barrier, Queue, Value ,Array.
- 常用方法与属性与multiprocessing中的一致

In [ ]:
# 使用Queue给Process进程做进程间通信
from multiprocessing import Process, Pool, Queue
import os
import time

def write(q):
    for i in range(4):
        q.put(i)
        print("子进程%s放入了%s" % (os.getpid(), i))

def read(q):
    while True:
        # 如果队列为空，get方法就会报错
        try:
            msg = q.get(block=False)
            print("子进程%s读取了%s" % (os.getpid(), msg))
        except:
            print("所有消息已经取出")
            break

if __name__ == "__main__":
    q = Queue(3)
    for i in range(3):
        pw = Process(target=write, args=(q,))
        pr = Process(target=read, args=(q,))
        pw.start()
        pr.start()

        pw.join()
        pr.join()
    print("主进程结束")



In [ ]:
# 消费者生产者模型
from multiprocessing import Process, Queue
import time
import os

def prodecer(q):
    for i in range(10):
        s = "厨师制作的" + str(i) + "号菜品"
        q.put(s)

def consumer(q):
    while True:
        time.sleep(0.2)
        try:
                ifo = q.get_nowait()
                print("{} 吃掉了 {}".format(os.getpid(), ifo))
        except:
            break

if __name__ == '__main__':
    print("主程序开始运行")
    q = Queue()
    p = Process(target=prodecer, args=(q, ))
    c = Process(target=consumer, args=(q, ))
    p.start()
    c.start()
    p.join()
    c.join()
    print("程序运行结束")
    
"""
#另外一种写法
def prodecer(q):
    for i in range(10):
        s = "厨师制作的" + str(i) + "号菜品"
        q.put(s)
    q.put(None)
def consumer(q):
    while True:
        time.sleep(0.2)
        ifo = q.get_nowait()
        if ifo is None:
            break
        print("{} 吃掉了 {}".format(os.getpid(), ifo))
"""

In [ ]:
# 多生产者多消费者模型，发送结束信号是个麻烦
from multiprocessing import Process, Queue
import time
import os

def prodecer(q, j):
    print("第{}个子进程开始生产".format(j+1))
    for i in range(10):
        s = "第"+ str(j+1) +"个进程制作的" + str(i+1) + "号菜品"
        q.put(s)
    print("这是进程{}".format(os.getpid()))
def consumer(q):
    while True:
        try:
            time.sleep(0.2)
            ifo = q.get_nowait()
            print("{} 吃掉了 {}".format(os.getpid(), ifo))
        # 下面语法是先让程序休眠，因为消费者可能比生产者跑得快，还没生产出来
        # 为了等一会生产者就休眠，一旦等到生产者生产完毕不再生产，
        # 消费者等了0.1秒后没反应就直接结束程序
        # 虽然能够结束程序，但是不好的点是休眠时间是自己设定的，
        # 时间设定的不好就会过早或过晚结束程序，造成错误或浪费资源
        except:
            time.sleep(0.1)
            if q.empty() is True:
                break
            else:
                pass

if __name__ == '__main__':
    print("主程序开始运行")
    q = Queue()
    pro, con = [], []
    for i in range(10):
        p = Process(target=prodecer, args=(q, i))
        c = Process(target=consumer, args=(q, ))
        p.start()
        c.start()
        pro.append(p)
        con.append(c)
    for p in pro:
        p.join()
    for c in con:
        c.join()
    print("程序运行结束")

In [2]:
# JoinableQueue举例实现多生产者多消费者模型
import multiprocessing
from multiprocessing import Process, JoinableQueue
import time, os


def producer(q, j):
    print("第{}个进程开始生产".format(j + 1))
    for i in range(10):
        s = "进程" + str(j+1) + "生产" + "第" + str(i + 1) + "个产品"
        q.put(s)
    print("进程{}的生产完毕".format(j + 1))
    # 生产完毕，使用此方法进行阻塞，直到队列中所有项目均被消费者消费完。
    # 等到收到消费者返还的task_done信息，新的进程才开始接着生产
    q.join()


def consumer(q):
    while True:
        ifo = q.get()
        print("进程{}吃掉了{}".format(os.getpid(), ifo))
        q.task_done()  # 向q.join()发送一次信号,表明队列中的所有项目已经被消费完毕


if __name__ == "__main__":
    print("主程序开始运行")
    p_l, c_l = [], []
    q = JoinableQueue()
    for i in range(5):
        p = Process(target=producer, args=(q, i))
        c = Process(target=consumer, args=(q,))
        # 如果不加守护，那么子进程永远结束不了，但是加了守护之后，
        # 必须确保生产者的内容生产完并且被处理完了，
        # 所以必须还要在主进程给生产者设置join，才能确保生产者生产的任务被执行完了
        # 并且能够确保守护进程在所有任务执行完成之后才随着主进程的结束而结束。
        c.daemon = True  # 或者在c中加入daemon=True,daemon必须要设置在start以前
        p.start()
        c.start()
        p_l.append(p)


    for p in p_l:
        p.join()
    # 不能再加c.join(),加了之后主程序无法结束
    print("程序运行完毕")

主程序开始运行
程序运行完毕


# 使用Queue给pool进程池做进程通信
- 如果要使用Pool创建进程，就需要使用multiprocessing.Manager()中的Queue()，而不是multiprocessing.Queue()，否则会报错，RuntimeError：Queue objects should not only be shared between proocesses through inheritance

In [ ]:
from multiprocessing import Process, Pool, Queue, Manager
import os
import time

def write(q):
    try:
        for i in range(100):
            q.put(i, block=False)  # 等价于q.put_nowait(i)
            print("子进程%s放入了%s" % (os.getpid(), i))
    except:
        print("队列已经放满了")

def read(q):
    while True:
        # 如果队列为空，get方法就会报错
        try:
            msg = q.get(block=False, timeout=1)
            print("子进程%s读取了%s" % (os.getpid(), msg))
        except:
            print("所有消息已经取出")
            break

if __name__ == "__main__":
    # q = Queue(3)  # 使用这个会报错
    q = Manager().Queue(100)
    pool = Pool(3)
    for i in range(5):
        pool.apply_async(func=write, args=(q, ))
        pool.apply_async(func=read, args=(q, ))

    pool.close()
    pool.join()
    print("主进程结束")



In [ ]:
# pipe实现生产者消费者模型
from multiprocessing import Process, Pipe
import os

def producer(seq, pipe):
    print("生产者开始生产")
    prd, con = pipe
    con.close()
    for i in seq:
        s = str(os.getpid()) + "发送红包" + str(i)
        prd.send(s)
        print(s)


def consumer(pipe):
    while True:
        try:
            prd, con = pipe
            prd.close()
            rv = con.recv()
            print("消费者接受了{}".format(rv))
        except EOFError: #当出现recv阻塞情况时退出，有点类似Queue中发送结束信号
            break

def main():
    seq = [i for i in range(10)]
    pro, con = Pipe()

    c = Process(target=consumer, args=((pro, con), ))
    c.start()
    producer(seq, (pro, con))
    pro.close()
    con.close()
    c.join()

if __name__ == '__main__':
    print("程序开始")
    main()
    print("程序结束")

In [ ]:
# pipe示例
import multiprocessing
import time

def consumer(output_p):
    while True:
        item = output_p.recv()
        print("item {} is received".format(item))


def producer(input_p):
    for item in range(10):
        input_p.send(item)


if __name__ == "__main__":
    (output_p, input_p) = multiprocessing.Pipe()
    producer(input_p)

    consumer_p = multiprocessing.Process(target=consumer, args=(output_p,  ))
    consumer_p.start()
    time.sleep(5)
    consumer_p.terminate()
    consumer_p.join()
    print("程序运行结束")


# 为什么多处理模块需要调用特定的 freeze_support模块才能“冻结”来生成Windows可执行文件？
- 原因是在Windows上缺少fork()(这是 not完全正确).因此,在Windows上,通过创建一个新的进程来模拟叉,其中正在运行在Linux上运行的代码(在子进程中运行).由于代码将在技术上无关的流程中运行,因此必须在代码运行之前交付代码.它的交付方式首先是被腌制,然后通过管道从原始流程发送到新的流程.此外,这个新进程被通知它必须运行通过管道传递的代码,通过将–multiprocessing-fork命令行参数传递给它.如果您查看了一个关于freeze_support()函数的 implementation,那么它的任务是检查它正在运行的进程是否运行通过管道传递的代码.

# 多进程之信号量Semaphore
- 互斥锁同时只允许一个线程更改数据，而信号量Semaphore是同时允许一定数量的线程更改数据 。
- 信号量同步基于内部计数器，每调用一次acquire()，计数器减1；每调用一次release()，计数器加1.当计数器为0时，acquire()调用被阻塞。这是迪科斯彻（Dijkstra）信号量概念P()和V()的Python实现。信号量同步机制适用于访问像服务器这样的有限资源。
- 和多线程Semaphore的用法很像

# 多进程之事件类
- python线程的事件(Event)用于主线程控制其他线程的执行，事件主要提供了三个方法 set、wait、clear。
    - 事件处理的机制：全局定义了一个“Flag”，如果“Flag”值为 False，那么当程序执行 event.wait 方法时就会阻塞，如果“Flag”值为True，那么event.wait 方法时便不再阻塞。
    - clear：将“Flag”设置为False
    - set：将“Flag”设置为True

In [ ]:
# multiprocessing.Manager示例

import multiprocessing


# 定义简单函数对不可变元素进行修改
def simple_test(id, test_dict):
    test_dict['name'] = id
    print(test_dict['name'])

# 定义复杂函数对可变对象进行修改
def complex_test(id, test_dict):
    """
    test_dict['name'] = ['jiaojiao', 'xiaoxiao']
    test_dict['name'][0] = id
    """
    # 以上方法无法对可变元素进行修改，只有赋值才能解决问题
    test_dict['name'] = ['jiaojiao', 'xiaoxiao']
    value = test_dict['name']
    value[0] = id
    test_dict['name'] = value

    print(test_dict)

if __name__ == "__main__":
    # windows中必须要有这句语句，否则程序报错
    multiprocessing.freeze_support()
    with multiprocessing.Manager() as manager:  # 使用with语句可自动给manager上锁，否则需自己上锁
        test_dict = manager.dict()
        test_dict['name'] = 'xiaoxiao'
        for i in range(10):
            p = multiprocessing.Process(target=complex_test, args=(i, test_dict))
            p.start()
        p.join()

In [ ]:
# Pool+ Manager
# multiprocessing.Manager示例

import multiprocessing
from multiprocessing import Pool

# 定义简单函数对不可变元素进行修改
def simple_test(id, test_dict):
    test_dict['name'] = id
    print(test_dict['name'])

# 定义复杂函数对可变对象进行修改
def complex_test(id, test_dict):
    """
    test_dict['name'] = ['jiaojiao', 'xiaoxiao']
    test_dict['name'][0] = id
    """
    # 以上方法无法对可变元素进行修改，只有赋值才能解决问题
    test_dict['name'] = ['jiaojiao', 'xiaoxiao']
    value = test_dict['name']
    value[0] = id
    test_dict['name'] = value

    print(test_dict)

if __name__ == "__main__":
    # windows中必须要有这句语句，否则程序报错
    multiprocessing.freeze_support()
    pool = Pool(4)
    with multiprocessing.Manager() as manager:  # 使用with语句可自动给manager上锁，否则需自己上锁
        test_dict = manager.dict()
        test_dict['name'] = 'xiaoxiao'
        for i in range(20):
            pool.apply_async(complex_test, args=(i, test_dict))
        pool.close()
        pool.join()

In [ ]:
# 信号量举例
from multiprocessing import Process, Semaphore
import time, random


def go_ktv(sem, user):
    sem.acquire()
    print('%s 占到一间ktv小屋' % user)
    time.sleep(random.randint(0, 3))  # 模拟每个人在ktv中待的时间不同
    sem.release()


if __name__ == '__main__':
    sem = Semaphore(4)
    p_l = []
    for i in range(13):
        p = Process(target=go_ktv, args=(sem, 'user%s' % i,))
        p.start()
        p_l.append(p)

    for i in p_l:
        i.join()
    print('============》')

In [ ]:
# 事件方法的使用
from multiprocessing import Event

e = Event()  # 创建一个事件对象
print(e.is_set())  # is_set()查看一个事件的状态，默认为False，可通过set方法改为True
print('look here！')
# e.set()          #将is_set()的状态改为True。
# print(e.is_set())#is_set()查看一个事件的状态，默认为False，可通过set方法改为Tr
# e.clear()        #将is_set()的状态改为False
# print(e.is_set())#is_set()查看一个事件的状态，默认为False，可通过set方法改为Tr
e.wait()  # 根据is_set()的状态结果来决定是否在这阻塞住，is_set()=False那么就阻塞，is_set()=True就不阻塞
print('give me！！')

# set和clear  修改事件的状态 set-->True   clear-->False
# is_set     用来查看一个事件的状态
# wait       依据事件的状态来决定是否阻塞 False-->阻塞  True-->不阻塞

In [1]:
l = ['1' for i in range(5)]
print(l)

['1', '1', '1', '1', '1']
